## What is Zero-Shot Learning?

- <span style="color:green"> **Zero-Shot Learning** </span> is a concept, that a model when trained on enough unlabeled data (unsupervised learning) is able to generalize/ recognize at inference time even though the model was not trained on the inference data. This can be used in NLP, Images etc.
- <span style="color:green"> **Zero-Shot Learning** </span> is a setup in which a model can learn to recognize things that it hasn’t explicitly seen before in training.

#### Load useful libraries and data

In [1]:
from transformers import pipeline
import pandas as pd
import numpy as np
from tqdm import tqdm


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
data = pd.read_csv(
    "data/SMSSpamCollection.txt",
    encoding="utf-8",
    header=None,
    delimiter="\t",
    names=["target", "text"],
)

In [3]:
data.head(5)

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
print(f'There are {data.shape[0]} rows in the dataset')

### Preparing the pipeline in one-line of code!

In [ ]:
classifier = pipeline("zero-shot-classification",device = 0)

### Making Predictions

This model works best with informative labels, spam/ham are not so informative. Using spam/ham leads to a Hamming loss of 53% vs using click bait/written by humans leading to 19%

Can you find better label descriptions?

In [ ]:
category_map = {"spam":"click bait", "ham":"written by humans"}

In [ ]:
candidate_labels = list(category_map.values())
predictedCategories = []
trueCategories = []
for i in tqdm(range(100)):
    text = data.iloc[i,]['text']
    cat = [data.iloc[i,]['target']]
    res = classifier(text, candidate_labels, multi_label=False)
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = next(k for i, (k,v) in enumerate(sorted_dict.items()))

    predictedCategories.append(categories)
    trueCats = [category_map[x] for x in cat]
    trueCategories.append(trueCats)

In [ ]:
for y_true, y_pred in zip(trueCategories[:3], predictedCategories[:3]):
    print(f'True Categories {y_true}')
    print(f'Predicted Categories {y_pred}')
    print('#'*50)

### Hamming Loss
The Hamming loss is the fraction of labels that are incorrectly predicted.

In [ ]:
from sklearn.metrics import hamming_loss
print(f'The hamming loss is {hamming_loss(trueCategories,predictedCategories):.4f} compared to 0.0237 from the last trained model in Notebook 1')

## Check your understanding

Can you find better "labels" for the category description improve the Hamming Loss?